In [ ]:
import torch
import pandas as pd
import numpy as np
import os, sys
import time

import matplotlib.pyplot as plt
sys.path.append(os.path.realpath('./src/'))
from materialEncoder import MaterialEncoder


from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon, Ellipse
from smallestEllipse import *

%matplotlib qt


In [ ]:
def check_xl():
    df = pd.read_excel('./data/newMaterialData_7D.xlsx')# solidworksMaterialDatabaseCost # aluminum
    return df

df = check_xl()
df.info()

In [ ]:
x = df['Rho (g/cc)/Rho0'].values
y = df['E(GPa)/E0'].values
txt = df['Material'].values
arr = np.array([0]*25 + [1]*25)
clrs = ['red' if val == 0 else 'blue' for val in arr]


print(txt)
fig, ax = plt.subplots()
ax.scatter(x, y, c=clrs)

for i, j in enumerate(txt):
    ax.annotate(j, (x[i], y[i]))
    
from smallestEllipse import *

for i in [0,1]:
    pts = np.array([x[25*i:25*(i+1)], y[25*i:25*(i+1)]]).T
    enclosing_ellipse = welzl(np.array(pts, dtype=float))
    # plot resulting ellipse
    center,a,b,t = enclosing_ellipse
    elli = plot_ellipse(enclosing_ellipse, str='k')
    ellipse = Ellipse(xy=center, width=2*a, height=2*b, angle=np.degrees(t), edgecolor='k', fc='r' if i == 0 else 'b', alpha=0.3, lw=2)
    ax.add_patch(ellipse)

ax.set_xlabel('Density')
ax.set_ylabel('Youngs Modulus')

In [ ]:
combined_array = np.column_stack((x, y))

# Convert the NumPy array to a PyTorch tensor
d = torch.tensor(combined_array, dtype=torch.float32)

min_vals = d.min(dim=0, keepdim=True)[0]  # Minimum values for each column
max_vals = d.max(dim=0, keepdim=True)[0]  # Maximum values for each column

# Apply normalization
data = (d - min_vals) / (max_vals - min_vals)

print(data)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Define the Encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(2, 50)
        self.fc21 = nn.Linear(50, 2)
        self.fc22 = nn.Linear(50, 2)

    def forward(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

# Define the Decoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc3 = nn.Linear(2, 50)
        self.fc4 = nn.Linear(50, 2)

    def forward(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

# Define the VAE
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        mu, logvar = self.encoder(x.view(-1, 2))
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Define the loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 2), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    beta = 5e-5
    return BCE + beta*KLD

# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Dummy dataset

# Training loop
model.train()
for epoch in range(1000):  # Number of epochs
    optimizer.zero_grad()
    recon_batch, mu, logvar = model(data)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Plotting the latent space
model.eval()

with torch.no_grad():
    _, mu, _ = model(data)
    plt, ax = plt.subplots()
    ax.scatter(mu[:, 0].cpu(), mu[:, 1].cpu(), c=clrs, marker='o')
    ax.set_title('Latent Space')
    ax.set_xlabel('Latent Dimension 1')
    ax.set_ylabel('Latent Dimension 2')
    for i,j in enumerate(txt):
        ax.annotate(j, (mu[i, 0].cpu().numpy(), mu[i, 1].cpu().numpy()))


for i in [0,1]:
    pts = np.array([mu[25*i:25*(i+1),0].cpu().numpy(), mu[25*i:25*(i+1),1].cpu().numpy()]).T
    enclosing_ellipse = welzl(np.array(pts, dtype=float))
    # plot resulting ellipse
    center,a,b,t = enclosing_ellipse
    elli = plot_ellipse(enclosing_ellipse, str='k')
    ellipse = Ellipse(xy=center, width=2*a, height=2*b, angle=np.degrees(t), edgecolor='k', fc='r' if i == 0 else 'b', alpha=0.3, lw=2)
    ax.add_patch(ellipse)



In [ ]:
plt.close('all')

In [ ]:
txt